In [ ]:
import tensorflow as tf
import numpy as np
import vgg19
import matplotlib.pyplot as plt
import utils

In [ ]:
#img = tf.placeholder(tf.float32, [None, 224, 224, 3])
i_max = 1000
i_print = 100

device = '/gpu:0'
show = True

In [ ]:
def gram_matrix(F, N, M):
    Ft = tf.reshape(F, (M, N))
    if M < N:
        return tf.matmul(Ft,tf.transpose(Ft))
    else:
        return tf.matmul(tf.transpose(Ft), Ft)

In [ ]:
def main_loss_function(x):
    return

In [ ]:
def create_noise_image(width, height):
    image = np.random.uniform(0.3,0.7, (1, height, width, 3)).astype('float32')
    
    if(show == True):
        plt.imshow(image.reshape(height,width,3))
        plt.show()
    return image

In [ ]:
def get_image(path):
    #load image
    img_input = utils.load_image(path)
    img = img_input.reshape((1, 224, 224, 3))
    
    #show image
    if(show == True):
        plt.imshow(img_input.reshape((224,224,3)))
        plt.show()
        
    return img

In [ ]:
def get_layers(image, scope):      
    #vgg
    vgg = vgg19.Vgg19()
    
    with tf.name_scope(scope):
        vgg.build(image)

    layers = [vgg.pool4, vgg.pool3, vgg.pool2, vgg.pool1, vgg.conv1_1]
    #layers = [vgg.conv1_1]
    
    return layers

In [ ]:
def get_gram_loss(target_layers, input_layers):
    gram_losses = []
    #go through layers
    for tl, il in zip(target_layers, input_layers):
        #get dimensions
        dimensions = tl.get_shape().as_list()
        size = dimensions[3]*dimensions[2]*dimensions[1]
        
        #get gram
        target_gram = gram_matrix(tl, dimensions[3], dimensions[2]*dimensions[1])
        input_gram = gram_matrix(il, dimensions[3], dimensions[2]*dimensions[1])
        #gram_losses.append(tf.nn.l2_loss(target_gram))
        #gram_losses.append(tf.nn.l2_loss(input_gram))
        
        #get losses
        gram_losses.append(tf.log(tf.reduce_sum(tf.square(target_gram - input_gram)) / size ))
    
    #get gram_loss
    gram_loss = tf.reduce_sum(gram_losses)
    
    return gram_losses

In [ ]:
def get_loss(target_layers, input_layers):
    loss = 0
    losses = []
    
    #gram loss
    gram_loss = get_gram_loss(target_layers, input_layers) 
    
    #get histogram matrices
    
    
    

    return gram_loss

In [ ]:
with tf.device(device):
    sess = tf.InteractiveSession()
        
    #placeholders
    input_image = tf.placeholder(tf.float32, [1, 224, 224, 3])
    
    #load style image
    input_img = get_image("./test_data/tiger.jpeg")
    #create target image
    target_img = (create_noise_image(224, 224))
        
    #fill feed dictionary
    feed = {input_image: input_img}
                 
    #create variables                 
    target = tf.Variable(target_img, tf.float32)
            
    #input vgg
    style_layers = get_layers(input_image, "style")
    #target vgg
    target_layers = get_layers(target, "target")
    
    #get losses
    loss = tf.reduce_sum(get_loss(target_layers, style_layers))
    


In [ ]:
    
    #handlers
    optimizer = tf.train.GradientDescentOptimizer(1.0)
    minimize = optimizer.minimize(loss)
    
    #init
    sess.run(tf.global_variables_initializer())
    
    
    print(sess.run(loss, feed_dict = feed))
    print(sess.run(get_loss(target_layers, style_layers), feed_dict = feed))
    
    #show image
    if(show == True):
        plt.imshow(sess.run(tf.reshape(target,(224,224,3)), feed_dict = feed))
        plt.show()
    

In [ ]:
    for i in range(i_max):
        
        #optimize step
        sess.run(minimize, feed_dict = feed)
        
        print(i, end = '')
        print(' ', end = '')
        
        if((i+1) % i_print == 0):
            
            #print loss
            #print(sess.run(target_layers, feed_dict = feed))
            print('')
            print(sess.run(loss, feed_dict = feed))
            print(sess.run(get_loss(target_layers, style_layers), feed_dict = feed))

            #show image
            if(show == True):
                plt.imshow(sess.run(tf.reshape(target,(224,224,3)), feed_dict = feed))
                plt.show()

    
